In [1]:
!which conda

/home/abhinavmishra/anaconda3/bin/conda


In [3]:
!which python

/home/abhinavmishra/anaconda3/bin/python


In [8]:
import pandas as pd
df = pd.read_parquet("green_tripdata_2025-10.parquet")

In [9]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-10-01 00:21:47,2025-10-01 00:24:37,N,1.0,247,69,1.0,0.70,5.80,...,0.5,1.70,0.00,NaN,1.0,10.00,1.0,1.0,0.00,0.00
1,2,2025-10-01 00:14:03,2025-10-01 00:24:14,N,1.0,66,25,1.0,1.61,11.40,...,0.5,2.78,0.00,NaN,1.0,16.68,1.0,1.0,0.00,0.00
2,2,2025-10-01 00:16:44,2025-10-01 00:16:47,N,5.0,244,244,1.0,0.00,10.00,...,0.0,2.20,0.00,NaN,1.0,13.20,1.0,2.0,0.00,0.00
3,2,2025-10-01 00:07:36,2025-10-01 00:32:14,N,1.0,95,170,1.0,10.37,43.60,...,0.5,11.31,6.94,NaN,1.0,67.85,1.0,1.0,2.75,0.75
4,2,2025-09-30 21:10:29,2025-09-30 21:22:30,N,1.0,82,138,1.0,4.07,19.80,...,0.5,6.82,0.00,NaN,1.0,34.12,1.0,1.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49411,6,2025-10-31 23:02:00,2025-11-01 00:28:33,None,NaN,241,61,NaN,20.09,2.90,...,0.5,0.00,6.94,NaN,0.3,63.84,NaN,NaN,NaN,0.00
49412,6,2025-10-31 23:51:34,2025-11-01 00:20:58,None,NaN,53,225,NaN,10.11,0.00,...,0.5,0.00,0.00,NaN,0.3,34.76,NaN,NaN,NaN,0.00
49413,2,2025-10-31 23:08:00,2025-10-31 23:42:00,None,NaN,7,170,NaN,4.20,45.14,...,0.5,10.03,0.00,NaN,1.0,60.17,NaN,NaN,NaN,0.75
49414,2,2025-10-31 23:45:00,2025-11-01 00:08:00,None,NaN,255,25,NaN,4.20,30.93,...,0.5,4.86,0.00,NaN,1.0,37.29,NaN,NaN,NaN,0.00


In [15]:
# calculating duration of trip
df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

In [16]:
df.duration

0       0 days 00:02:50
1       0 days 00:10:11
2       0 days 00:00:03
3       0 days 00:24:38
4       0 days 00:12:01
              ...      
49411   0 days 01:26:33
49412   0 days 00:29:24
49413   0 days 00:34:00
49414   0 days 00:23:00
49415   0 days 00:14:00
Name: duration, Length: 49416, dtype: timedelta64[us]

In [30]:
df['duration'] = df['duration'].apply(lambda x : x.total_seconds())

In [31]:
df['duration']

0         170.0
1         611.0
2           3.0
3        1478.0
4         721.0
          ...  
49411    5193.0
49412    1764.0
49413    2040.0
49414    1380.0
49415     840.0
Name: duration, Length: 49416, dtype: float64

In [33]:
df['duration'] = df['duration']/60

In [34]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,duration
0,2,2025-10-01 00:21:47,2025-10-01 00:24:37,N,1.0,247,69,1.0,0.70,5.80,...,1.70,0.00,NaN,1.0,10.00,1.0,1.0,0.00,0.00,2.833333
1,2,2025-10-01 00:14:03,2025-10-01 00:24:14,N,1.0,66,25,1.0,1.61,11.40,...,2.78,0.00,NaN,1.0,16.68,1.0,1.0,0.00,0.00,10.183333
2,2,2025-10-01 00:16:44,2025-10-01 00:16:47,N,5.0,244,244,1.0,0.00,10.00,...,2.20,0.00,NaN,1.0,13.20,1.0,2.0,0.00,0.00,0.050000
3,2,2025-10-01 00:07:36,2025-10-01 00:32:14,N,1.0,95,170,1.0,10.37,43.60,...,11.31,6.94,NaN,1.0,67.85,1.0,1.0,2.75,0.75,24.633333
4,2,2025-09-30 21:10:29,2025-09-30 21:22:30,N,1.0,82,138,1.0,4.07,19.80,...,6.82,0.00,NaN,1.0,34.12,1.0,1.0,0.00,0.00,12.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49411,6,2025-10-31 23:02:00,2025-11-01 00:28:33,None,NaN,241,61,NaN,20.09,2.90,...,0.00,6.94,NaN,0.3,63.84,NaN,NaN,NaN,0.00,86.550000
49412,6,2025-10-31 23:51:34,2025-11-01 00:20:58,None,NaN,53,225,NaN,10.11,0.00,...,0.00,0.00,NaN,0.3,34.76,NaN,NaN,NaN,0.00,29.400000
49413,2,2025-10-31 23:08:00,2025-10-31 23:42:00,None,NaN,7,170,NaN,4.20,45.14,...,10.03,0.00,NaN,1.0,60.17,NaN,NaN,NaN,0.75,34.000000
49414,2,2025-10-31 23:45:00,2025-11-01 00:08:00,None,NaN,255,25,NaN,4.20,30.93,...,4.86,0.00,NaN,1.0,37.29,NaN,NaN,NaN,0.00,23.000000
